In [ ]:
# IMPORTS
import sys
import os
import glob
from pathlib import Path

import plotbot
from plotbot import *
from plotbot.data_classes.data_types import data_types, get_local_path
from plotbot.data_download_pyspedas import download_spdf_data

print("✅ Imports complete")

In [ ]:
# EPAD DATA PATH DIAGNOSTIC
# Run this cell and copy ALL output to share

print("="*80)
print("EPAD DATA PATH DIAGNOSTIC")
print("="*80)

# Test parameters
trange = ['2020-01-28/00:00:00', '2020-01-29/00:00:00']
data_type = 'spe_sf0_pad'

print(f"\nTest time range: {trange}")
print(f"Data type: {data_type}")

# 1. Configuration
print(f"\n{'='*80}")
print("1. PLOTBOT CONFIGURATION")
print(f"{'='*80}")
print(f"config.data_dir:        {config.data_dir}")
print(f"config.data_server:     {config.data_server}")
print(f"SPEDAS_DATA_DIR:        {os.environ.get('SPEDAS_DATA_DIR', 'NOT SET')}")
print(f"Current working dir:    {os.getcwd()}")

# 2. Expected paths
print(f"\n{'='*80}")
print("2. EXPECTED EPAD DATA PATHS")
print(f"{'='*80}")
spe_config = data_types.get(data_type)
raw_path = spe_config['local_path']
computed_path = get_local_path(data_type)
data_level = spe_config['data_level']
formatted_path = computed_path.format(data_level=data_level)
year_2020_path = os.path.join(formatted_path, '2020')

print(f"Raw local_path:         {raw_path}")
print(f"Computed path:          {computed_path}")
print(f"Formatted (l3):         {formatted_path}")
print(f"2020 directory:         {year_2020_path}")
print(f"Does 2020 dir exist?    {os.path.exists(year_2020_path)}")

# 3. Search for any existing epad files
print(f"\n{'='*80}")
print("3. SEARCHING FOR EXISTING EPAD FILES")
print(f"{'='*80}")
base_dir = os.path.dirname(config.data_dir)
search_patterns = [
    os.path.join(config.data_dir, '**', '*spe*sf0*pad*.cdf'),
    os.path.join(base_dir, 'psp_data', '**', '*spe*.cdf'),  # Legacy
]

found_files = set()
for pattern in search_patterns:
    files = glob.glob(pattern, recursive=True)
    found_files.update(files)

if found_files:
    print(f"Found {len(found_files)} existing epad-related file(s):")
    for f in sorted(found_files)[:10]:
        print(f"  - {f}")
    if len(found_files) > 10:
        print(f"  ... and {len(found_files) - 10} more")
else:
    print("❌ No existing epad CDF files found")

# 4. Test PySpedas direct download
print(f"\n{'='*80}")
print("4. TESTING PYSPEDAS DOWNLOAD")
print(f"{'='*80}")
try:
    import pyspedas
    print(f"PySpedas imported successfully")
    print(f"Calling pyspedas.psp.spe() for {trange}...\n")
    
    result = pyspedas.psp.spe(
        trange=trange,
        datatype='spe_sf0_pad',
        level='l3',
        no_update=False,
        downloadonly=True,
        notplot=True,
        get_support_data=True
    )
    
    print(f"PySpedas result type:   {type(result)}")
    print(f"PySpedas result value:  {result}")
    print(f"Is truthy:              {bool(result)}")
    print(f"Length:                 {len(result) if result else 0}")
    
    if result and len(result) > 0:
        print(f"\nFiles returned by PySpedas:")
        for i, fpath in enumerate(result):
            abs_path = os.path.abspath(fpath)
            exists = os.path.exists(fpath)
            print(f"  [{i}] {fpath}")
            print(f"       Absolute: {abs_path}")
            print(f"       Exists: {exists}")
            if exists:
                size_kb = os.path.getsize(fpath) / 1024
                print(f"       Size: {size_kb:.1f} KB")
    else:
        print(f"\n❌ PySpedas returned empty/None - data not available on SPDF")
        print(f"   This SHOULD trigger Berkeley fallback in dynamic mode")
        
except Exception as e:
    print(f"❌ Error during PySpedas download: {e}")
    import traceback
    traceback.print_exc()

# 5. Test Plotbot's wrapper function
print(f"\n{'='*80}")
print("5. TESTING PLOTBOT'S download_spdf_data()")
print(f"{'='*80}")
print(f"Calling download_spdf_data()...\n")

plotbot_result = download_spdf_data(trange, data_type)

print(f"\nPlotbot result type:    {type(plotbot_result)}")
print(f"Plotbot result value:   {plotbot_result}")
print(f"Is truthy:              {bool(plotbot_result)}")
print(f"Length:                 {len(plotbot_result) if plotbot_result else 0}")

if not plotbot_result:
    print(f"\n⚠️  CRITICAL: Plotbot's download_spdf_data() returned falsy!")
    print(f"    In dynamic mode, this SHOULD trigger Berkeley fallback")
    print(f"    If Berkeley fallback didn't happen, that's the bug!")

# 6. Check if files exist in 2020 directory specifically
print(f"\n{'='*80}")
print("6. CHECKING 2020 DIRECTORY CONTENTS")
print(f"{'='*80}")
if os.path.exists(year_2020_path):
    files_in_2020 = os.listdir(year_2020_path)
    print(f"Files in 2020 directory: {len(files_in_2020)}")
    # Look for the specific date we're testing
    jan28_files = [f for f in files_in_2020 if '20200128' in f]
    if jan28_files:
        print(f"\n✅ Found {len(jan28_files)} file(s) for 2020-01-28:")
        for f in jan28_files:
            print(f"   - {f}")
    else:
        print(f"\n❌ No files found for 2020-01-28 in directory")
        print(f"   Files present (first 10):")
        for f in sorted(files_in_2020)[:10]:
            print(f"   - {f}")
else:
    print(f"❌ 2020 directory doesn't exist: {year_2020_path}")

# 7. Test import_data_function pattern matching
print(f"\n{'='*80}")
print("7. TESTING FILE PATTERN MATCHING")
print(f"{'='*80}")
file_pattern_import = spe_config['file_pattern_import']
print(f"file_pattern_import: {file_pattern_import}")
formatted_pattern = file_pattern_import.format(data_level=data_level, date_str='20200128')
print(f"Formatted for date:  {formatted_pattern}")
print(f"\nConverting glob pattern to regex...")
import re
pattern_for_re = formatted_pattern.replace('*', '.*')
regex = re.compile(pattern_for_re, re.IGNORECASE)
print(f"Regex pattern:       {pattern_for_re}")

if os.path.exists(year_2020_path):
    print(f"\nTesting pattern against files in directory:")
    matches = []
    for fname in os.listdir(year_2020_path):
        if '20200128' in fname:  # Only test Jan 28 files
            is_match = bool(regex.match(fname))
            print(f"   {fname}: {'✅ MATCH' if is_match else '❌ NO MATCH'}")
            if is_match:
                matches.append(fname)
    print(f"\nTotal matches: {len(matches)}")

# 8. Test actual import_data_function logic
print(f"\n{'='*80}")
print("8. TESTING import_data_function FILE SEARCH LOGIC")
print(f"{'='*80}")
from plotbot.data_import import import_data_function

print(f"Calling import_data_function() with trange: {trange}")
print(f"This simulates what happens when plotbot tries to load the data\\n")

try:
    imported_data = import_data_function(trange, data_type)
    print(f"\\nResult type: {type(imported_data)}")
    print(f"Result is None: {imported_data is None}")
    if imported_data is not None:
        print(f"✅ import_data_function successfully loaded data!")
        if hasattr(imported_data, 'datetime_array'):
            dt_len = len(imported_data.datetime_array) if imported_data.datetime_array is not None else 0
            print(f"   datetime_array length: {dt_len}")
        if hasattr(imported_data, 'raw_data'):
            print(f"   raw_data keys: {list(imported_data.raw_data.keys()) if imported_data.raw_data else 'None'}")
    else:
        print(f"❌ import_data_function returned None!")
        print(f"   This means it couldn't find or load the CDF files")
except Exception as e:
    print(f"❌ Error during import_data_function: {e}")
    import traceback
    traceback.print_exc()

# 9. Summary
print(f"\n{'='*80}")
print("9. DIAGNOSIS SUMMARY")
print(f"{'='*80}")
print(f"\n✅ Expected 2020 path: {year_2020_path}")
print(f"✅ PySpedas returned: {'SUCCESS' if result else 'FAILURE/EMPTY'}")
print(f"✅ Plotbot wrapper returned: {'SUCCESS' if plotbot_result else 'FAILURE/EMPTY'}")
print(f"✅ Existing files found: {len(found_files)}")

print(f"\n📋 LIKELY ISSUE:")
if not result and not plotbot_result:
    print(f"   → Data not available on SPDF (embargo or removed)")
    print(f"   → Berkeley fallback should have triggered but didn't")
    print(f"   → Check get_data() logic around line 415-418")
elif result and not any(os.path.exists(f) for f in result):
    print(f"   → PySpedas claims success but files don't exist")
    print(f"   → Path mismatch between PySpedas and Plotbot")
elif os.path.exists(year_2020_path) and not jan28_files:
    print(f"   → 2020 directory exists but no Jan 28 files")
    print(f"   → Data never downloaded from SPDF or Berkeley")
elif os.path.exists(year_2020_path) and jan28_files and not matches:
    print(f"   → Files exist but pattern matching failed!")
    print(f"   → Check file_pattern_import vs actual filename case")
else:
    print(f"   → Check output above for specific issue")

print(f"\n{'='*80}")
print("COPY ALL OUTPUT ABOVE AND SHARE")
print(f"{'='*80}")